In [1]:
import hopsworks

In [2]:
from utils.feature_engineering import get_latest_sold_products
get_latest_sold_products()[0]

{'badges': [],
 'bumped_at': '2025-01-02T19:18:19.459Z',
 'buynow': True,
 'category_path_size': 'bottoms.denim.32',
 'category_path': 'bottoms.denim',
 'category_size': 'bottoms.32',
 'category': 'bottoms',
 'color': 'black',
 'condition': 'is_used',
 'cover_photo': {'id': 523731374,
  'listing_id': 72901253,
  'image': None,
  'created_at': '2025-01-02T19:18:19.417Z',
  'updated_at': '2025-01-02T19:18:19.469Z',
  'photoable_id': 72901253,
  'photoable_type': 'Listing',
  'width': None,
  'height': None,
  'url': 'https://media-assets.grailed.com/prd/listing/temp/858a575a33ce4d9eb30ecf5cd2fd9b91',
  'position': 0,
  'rotate': 0,
  'image_url': 'https://media-assets.grailed.com/prd/listing/temp/858a575a33ce4d9eb30ecf5cd2fd9b91'},
 'created_at_i': 1735845509,
 'created_at': '2025-01-02T19:18:29.301Z',
 'deleted': False,
 'department': 'menswear',
 'description': 'Distressed throughout, see pics\n505 model\nGrey/black wash\n\nONLY SHIPPING TO CANADA AND USA',
 'designer_names': "Levi's ×

In [3]:
from dotenv import load_dotenv
load_dotenv()
    
project = hopsworks.login()

2025-01-06 12:12:39,714 INFO: Initializing external client
2025-01-06 12:12:39,716 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 12:12:41,670 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159324


## Get data

### Quick data exploration
Taken from https://graildient-descent.streamlit.app/eda, using 10,000 samples.
We could do the same using the larger dataset.

#### Numerical/quantitative features
Target variable (sold_price):
- Most sold items are between 35-135$ (consider plotting bins) - consider outliers, since we are far from normal distribution, maybe try a log transformation?

Number of photos:
- Another numerical feature, could be added
- Price increases until 13 photos, then inconsistent

#### Categorical/qualitative features
- high-cardinality: designer, color, subcategory, size
- low-cardinality: category, condition

Target encoding more fitting for high-cardinality features. Whereas low-cardinality features could be one-hot encoded.

- designer a strong predictor
- we skip department, to focus on men's clothing only (it has better representation)
- there is good variation of sold prices in different subcategories => probably a good indicator
- could be interesting to use embeddings for color instead
- condition - perfect for ordinal encoding. The better the condition, the higher the average sold price.

##### Text
- title/description - should we do any pre-processing?
- we could look at sentiment and similar text analysis approaches

#### Images
- could be interesting to add embedding representation of title image

## Feature processing

In [4]:
from utils.feature_engineering import pipeline
df = pipeline(no_of_hits=2000)
df.shape

embedding designer names
embedding descriptions
embedding titles
embedding hashtags
embedding size
shape: (1_000, 12)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id       ┆ sold_at   ┆ designer_ ┆ descripti ┆ … ┆ size      ┆ color     ┆ followern ┆ sold_pric │
│ ---      ┆ ---       ┆ names     ┆ on        ┆   ┆ ---       ┆ ---       ┆ o         ┆ e         │
│ i64      ┆ datetime[ ┆ ---       ┆ ---       ┆   ┆ list[f32] ┆ i64       ┆ ---       ┆ ---       │
│          ┆ μs]       ┆ list[f32] ┆ list[f32] ┆   ┆           ┆           ┆ i64       ┆ i64       │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 72901253 ┆ 2025-01-0 ┆ [-0.06294 ┆ [-0.00694 ┆ … ┆ [0.0,     ┆ 146781285 ┆ 1         ┆ 40        │
│          ┆ 6 11:09:3 ┆ ,         ┆ 1,        ┆   ┆ 0.0, …    ┆           ┆           ┆           │
│          ┆ 3.919     ┆ 0.025376, ┆ 0.023401, ┆   ┆ 0.0]      ┆          

(1000, 12)

## Save data

In [5]:
fs = project.get_feature_store() 

In [6]:
grailed_items_fg = fs.get_or_create_feature_group(
    name='draft_grailed_items',
    description='Sold Grailed items',
    version=4,
    primary_key=['id'],
    event_time="sold_at",
    # expectation_suite=aq_expectation_suite
)

In [7]:
grailed_items_fg.insert(df)

Uploading Dataframe: 100.00% |█████████████| Rows 1000/1000 | Elapsed Time: 00:04 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/id2223_assignment1/Resources/jobs/draft_grailed_items_4_offline_fg_materialization/config_1736098402159) to trigger the materialization job again.


(Job('draft_grailed_items_4_offline_fg_materialization', 'SPARK'), None)

In [8]:
# TODO: Update feature description